In [19]:
from state_transformation_base import StateTransformationBaseTest, RUN_TIME, ONE_HOUR
from pytezos import MichelsonRuntimeError

self = StateTransformationBaseTest()
self.setUp()

self.current_time = RUN_TIME
self.id = len(self.storage['events'])

# Creating event:
event_params = self.default_event_params.copy()
event_params.update({
    'liquidityPercent': 10_000,
    'betsCloseTime': RUN_TIME + ONE_HOUR,
    'measurePeriod': ONE_HOUR,
})
self.storage = self.check_new_event_succeed(
    event_params=event_params,
    amount=self.measure_start_fee + self.expiration_fee)

In [20]:
# Participant A: adding liquidity 1/1 just at start:
self.storage = self.provide_liquidity(
    participant=self.a,
    amount=2_000_000,
    expected_for=1,
    expected_against=1)

In [21]:
# Participant B: bets FOR for 1 tez:
self.storage = self.bet(
    participant=self.b,
    amount=1_000_000,
    bet='for',
    minimal_win=1_000_000)

In [22]:
self.storage['events'][0]['winAgainstProfitLossPerShare'] / 100_000_000

1.0

In [23]:
self.storage['events'][0]['totalLiquidityAgainstSharesSum']

1000000

In [24]:
self.storage['events'][0]['betsAgainstLiquidityPoolSum']

500000

In [25]:
self.storage['events'][0]['betsForLiquidityPoolSum']

2000000

In [26]:
# Participant A: adding more liquidity after 30 mins with 1:4:
self.current_time = int(RUN_TIME + 0.5*ONE_HOUR)
self.storage = self.provide_liquidity(
    participant=self.a,
    amount=2_500_000,
    expected_for=4,
    expected_against=1)

In [27]:
def monitor_storage(storage):
    pass

In [28]:
self.storage['events'][0]['winAgainstProfitLossPerShare'] / 100_000_000

1.0

In [29]:
self.storage['events'][0]['totalLiquidityForSharesSum']

2000000

In [30]:
self.storage['events'][0]['totalLiquidityAgainstSharesSum']

1250000

In [31]:
self.storage['events'][0]['totalLiquidityForSharesSum']

2000000

In [32]:
self.storage['events'][0]['winAgainstProfitLossPerShare'] / 100_000_000

1.0

In [33]:
self.storage['winAgainstProfitLossPerShareAtEntry'][self.a, 0]

1000000

In [34]:
self.storage['liquidityForSharesLedger']

{('tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ', 0): 2000000}

In [35]:
# Participant C: adding more liquidity after 30 mins with 1:4:
self.storage = self.provide_liquidity(
    participant=self.c,
    amount=5_000_000,
    expected_for=4,
    expected_against=1)

In [36]:
newForShares = 2_000_000

In [37]:
self.storage['liquidityForSharesLedger'][(self.c, 0)]

2000000

In [44]:
event = self.storage['events'][0]
event['winAgainstProfitLossPerShare'] * newForShares / 100_000_000

2000000.0

In [40]:
self.storage['winAgainstProfitLossPerShareAtEntry'][(self.c, 0)]

100000000

In [41]:
self.storage['events'][0]['totalLiquidityForSharesSum']

4000000

In [42]:
self.storage['events'][0]['totalLiquidityAgainstSharesSum']

1750000

In [20]:
# Participant D: bets AGAINST for 2 tez:
self.storage = self.bet(
    participant=self.d,
    amount=2_000_000,
    bet='against',
    minimal_win=2_000_000)

In [22]:
self.storage['events'][0]['winAgainstProfitLossPerShare'] / 100_000_000

0.01

In [23]:
# Participant B: bets FOR for 4 tez:
self.storage = self.bet(
    participant=self.b,
    amount=4_000_000,
    bet='for',
    minimal_win=4_000_000)

In [24]:
self.storage['events'][0]['winAgainstProfitLossPerShare'] / 100_000_000

1.01

In [25]:
# Participant A: adding more liquidity after 30 mins with 1:4:
self.storage = self.provide_liquidity(
    participant=self.a,
    amount=5_000_000,
    expected_for=4,
    expected_against=1)

In [26]:
self.storage['events'][0]['totalLiquidityForSharesSum']

6000000

In [27]:
# Running measurement:
self.current_time = RUN_TIME + 2*ONE_HOUR
self.storage = self.check_start_measurement_succeed(sender=self.a)

# Emulating callback:
callback_values = {
    'currencyPair': self.currency_pair,
    'lastUpdate': self.current_time - int(0.5*ONE_HOUR),
    'rate': 8_000_000
}
self.storage = self.check_start_measurement_callback_succeed(
    callback_values=callback_values,
    source=self.a,
    sender=self.oracle_address)

# Closing event:
self.current_time = RUN_TIME + 3*ONE_HOUR
self.storage = self.close(sender=self.b)

# Emulating calback with price is increased 25%:
callback_values = {
    'currencyPair': self.currency_pair,
    'lastUpdate': self.current_time - int(0.5*ONE_HOUR),
    'rate': 6_000_000
}
self.storage = self.check_close_callback_succeed(
    callback_values=callback_values,
    source=self.b,
    sender=self.oracle_address)

In [39]:
profitLossPerShareAtEntry = self.storage['winAgainstProfitLossPerShareAtEntry'][(self.c, 0)]
profitLossPerShareAtEntry / 1_000_000

100.0

In [40]:
totalForShares = self.storage['liquidityForSharesLedger'][(self.c, 0)]
totalForShares

2000000

In [41]:
event = self.storage['events'][0]
(event['winAgainstProfitLossPerShare'] * totalForShares) / event['sharePrecision'] - profitLossPerShareAtEntry

-97980000.0

In [35]:
self.storage = self.withdraw(self.b, 0)


In [36]:
self.storage = self.check_withdraw_succeed(self.c, 5_020_000)


MichelsonRuntimeError: ('IF_LEFT', 'IF_LEFT', 'IF_LEFT', 'IF', 'SUB', 'mutez', 'expected natural number, got -92980000')

In [ ]:
# Withdrawals:
self.current_time = RUN_TIME + 4*ONE_HOUR
import pdb; pdb.set_trace()
self.storage = self.check_withdraw_succeed(self.a, 10_520_000)
self.storage = self.check_withdraw_succeed(self.b, 0)
self.storage = self.check_withdraw_succeed(self.c, 5_020_000)
self.storage = self.check_withdraw_succeed(self.d, 3_960_000)
